# RODD disturbance model estimation using Variational Inference with Pyro

In [1]:
import os
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import (SVI, TraceEnum_ELBO, config_enumerate, 
                        infer_discrete)
from pyro.infer.autoguide import AutoNormal

smoke_test = ('CI' in os.environ)

In [2]:
from platform import python_version
python_version()

'3.9.9'

In [3]:
pyro.set_rng_seed(101)
pyro.__version__

'1.8.0'

In [4]:
plot_dir = 'plots'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

## Inferring the parameters of a stochastic process with a discrete random decision variable

Suppose we have a noisy measurement of a disturbance process that looks like a series of randomly-occurring step changes:

<img src="images/Wong_and_Lee_fig1c_steps.png" width="320">

<center> Figure source: Wong and Lee (2009)</center>

We could generate a disturbance like this by integrating a 'random shock' variable which is sampled from either of two normal distributions, according to a discrete random variable $\gamma(k)$ which is either 0 or 1:

$$
\gamma(k) \sim
\begin{cases}
0 & \text{with probability } 1-\epsilon \\
1 & \text{with probability } \epsilon
\end{cases}
$$

where $\epsilon<<1$.

<img src="images/shock_dist_1.png" width="320">
<img src="images/shock_dist_2.png" width="320">

In general, the discrete-time model for this class of disturbances is:

$$p(k) = \frac{B(q^{-1})}{A(q^{-1})} w_p(k)$$

This is known as a randomly-occurring deterministic disturbance (RODD) (MacGregor et al. 1984).

Reference:

- MacGregor, J.F., Harris, T.J., and Wright, J.D. (1984). Duality Between the Control of Processes Subject to Randomly Occurring Deterministic Disturbances and ARIMA Stochastic Disturbances. Technometrics, 26(4), 389–397.
- Wong, W.C. and Lee, J.H. (2009). Realistic disturbance modeling using hidden Markov models: Applications in model-based process control. Journal of Process
Control, 19(9), 1438–1450.

## Data model

First, we construct a model of our belief about the process that generated the data:

<img src="images/RODD_step_diag.png" width="320">

Random shock variable

$$
w_{p}(k) \sim
\begin{cases}
\mathcal{N}(0, \sigma_{w_p}^2) & \text{with probability } 1-\epsilon \\
\mathcal{N}(0, b^2 \sigma_{w_p}^2) & \text{with probability } \epsilon
\end{cases}
$$

Disturbance generating process

$$p(k)=\frac{1}{1-z^{-1}}w_{p}(k)$$

Measurement noise

$$ v(k) \sim \mathcal{N}(0, \sigma_{M}^2)$$

or

$$ y_m(k) \sim \mathcal{N}(p(k), \sigma_{M}^2)$$

Suppose we have initial guesses of the shock probability $\epsilon$, the variance of the signal when there is no shock $\sigma_{w_p}^2$, and when a shock occurs $b^2\sigma_{w_p}^2$:

In [5]:
epsilon_guess = 0.05
b_guess = 50
scale_guess = 0.03

## 1.  Define a stochastic function

 - See: http://pyro.ai/examples/intro_part_i.html

### Random Shock Model

In [6]:
def random_shock(scale_guess, b_guess, epsilon_guess):
    # Scale, b and epsilon are always positive
    scale = pyro.sample("scale", dist.LogNormal(np.log(scale_guess), 0.25))
    b = pyro.sample("b", dist.LogNormal(np.log(b_guess), 0.25))
    epsilon = pyro.sample("epsilon", dist.LogNormal(np.log(epsilon_guess), 0.25))
    alpha = pyro.sample("alpha", dist.Bernoulli(epsilon))
    alpha = alpha.long()
    wp_dist = dist.Normal(torch.tensor([0.0, 0.0])[alpha], 
                          torch.tensor([scale, scale*b])[alpha])
    measurement = pyro.sample('obs', wp_dist)
    return measurement

def infrequent_step_5(scale_guess, b_guess, epsilon_guess):
    y = 0  # initial value
    measurements = []
    for k in range(5):
        y = y + random_shock(scale_guess, b_guess, epsilon_guess)
        measurements.append(y)
    return torch.tensor(measurements)


### Sampling from the model

In [7]:
# Draw random samples
for _ in range(5):
    print(infrequent_step_5(scale_guess, b_guess, epsilon_guess))

tensor([0.0156, 0.0179, 0.0519, 0.0739, 0.0711])
tensor([ 1.7345e-04, -4.2217e-03, -6.6964e-05,  1.6579e-02, -3.5251e-02])
tensor([ 0.0012, -0.0081,  0.0535, -0.2341, -0.1900])
tensor([ 0.0189, -0.0123,  0.0166,  0.0029, -0.0133])
tensor([-0.0015,  0.0066,  0.0482,  0.1065,  0.1311])


## Model conditioned on data

In [8]:
def infrequent_step_5_conditioned(scale_guess, b_guess, epsilon_guess, observations):
    y = 0  # initial value
    for k, y_m in enumerate(observations):
        y = y + random_shock(scale_guess, b_guess, epsilon_guess)
        pyro.sample(f"y_{k}", y, obs=y_m)
        print(f"  x_{t}.shape = {x.shape}")

## Generate a data set

In [9]:
# Generate random samples
data = []
for _ in range(10):
    data.append(infrequent_step_5(scale_guess, b_guess, epsilon_guess))
data

[tensor([-0.0155,  0.0770,  0.0630,  0.0503,  0.0369]),
 tensor([ 0.0111,  0.0054, -0.0019,  0.0091,  0.0018]),
 tensor([-0.0450, -0.0441, -0.0080, -0.0180, -0.0045]),
 tensor([ 0.0015, -0.0289, -0.0335, -0.0269, -0.0961]),
 tensor([0.0211, 0.0777, 0.1038, 0.1159, 0.1292]),
 tensor([-0.0375, -0.0661, -0.0248, -0.0266, -0.0020]),
 tensor([-0.0628, -0.0778, -0.0595, -0.0324, -0.0163]),
 tensor([ 0.0281, -0.0079,  0.0197,  0.0217, -0.0348]),
 tensor([0.0153, 0.1283, 0.1319, 0.1480, 0.1641]),
 tensor([0.0356, 0.0199, 0.0267, 0.0146, 0.0140])]

In [10]:
hmm_guide = AutoNormal(pyro.poutine.block(infrequent_step_5, expose=["scale", "b", "epsilon"]))

pyro.clear_param_store()
elbo = TraceEnum_ELBO(max_plate_nesting=1)
elbo.loss(infrequent_step_5, hmm_guide, data, data_dim=data_dim);

NameError: name 'data_dim' is not defined

## Looking for help online

Relevant or related topics/posts:

- [What is the difference between a particle filter (sequential Monte Carlo) and a Kalman filter?](https://stats.stackexchange.com/questions/2149/what-is-the-difference-between-a-particle-filter-sequential-monte-carlo-and-a)

> Note that Kalman filters by design only deal with Gaussian posterior distributions. Note that the different flavors (extended, unscented, ensemble) just vary in how they estimate the Gaussian in case of nonlinear dynamic/observation models. Particle filters can handle arbitrary arbitrary posteriors, including multi-modal ones. – GeoMatt22  Aug 31 '16.

- [Estimating parameters of a dynamic linear model](https://stats.stackexchange.com/questions/4991/estimating-parameters-of-a-dynamic-linear-model)

> If you have time varying parameters and want to do things sequentially (filtering), then SMC makes the most sense. MCMC is better when you want to condition on all of the data, or you have unknown static parameters that you want to estimate. Particle filters have issues with static parameters (degeneracy). Darren Wilkinson, Dec 14 '10.

- [From Dan Simon's "Optimal State Estimation":](https://stats.stackexchange.com/a/2153/226254)

> In a linear system with Gaussian noise, the Kalman filter is optimal. In a system that is nonlinear, the Kalman filter can be used for state estimation, but the particle filter may give better results at the price of additional computational effort. In a system that has non-Gaussian noise, the Kalman filter is the optimal linear filter, but again the particle filter may perform better. The unscented Kalman filter (UKF) provides a balance between the low computational effort of the Kalman filter and the high performance of the particle filter.

> The particle filter has some similarities with the UKF in that it transforms a set of points via known nonlinear equations and combines the results to estimate the mean and covariance of the state. However, in the particle filter the points are chosen randomly, whereas in the UKF the points are chosen on the basis of a specific algorithm*. Because of this, the number of points used in a particle filter generally needs to be much greater than the number of points in a UKF. Another difference between the two filters is that the estimation error in a UKF does not converge to zero in any sense, but the estimation error in a particle filter does converge to zero as the number of particles (and hence the computational effort) approaches infinity.

> *The unscented transformation is a method for calculating the statistics of a random variable which undergoes a nonlinear transformation and uses the intuition (which also applies to the particle filter) that it is easier to approximate a probability distribution than it is to approximate an arbitrary nonlinear function or transformation. See also this as an example of how the points are chosen in UKF."

## Generate a dataset

In [ ]:
# Parameters
true_scale = 0.01
true_b = 100
true_epsilon = 0.01

wp_measurements = [random_shocks(true_scale, true_b, true_epsilon).item()
                   for _ in range(1000)]

x_minmax = true_scale * true_b * np.array([-1 , 1])

plt.hist(wp_measurements, bins=np.linspace(*x_minmax, 51), 
         density=True)
plt.xlim(x_minmax)
plt.xlabel('Randomly occuring shocks')
plt.ylabel('Frequency')
plt.grid()
plt.savefig('plots/wp_samples.png')
plt.show()

In [ ]:
# Choose how much data to provide
data = torch.tensor(wp_measurements)

## 2. Variational Inference

### Guide function

The guide function represents the family of distributions we want to consider as our posterior distribution. In this case, it is obvious that the distribution is a Gaussian mixture model with 2 components.

The function below is based on example code here: https://pyro.ai/examples/gmm.html

This model below allows different scale for each mixture.

In [ ]:
K = 2  # Number of mixture components (fixed)

@config_enumerate
def model(data):
    # Global variables.
    weights = pyro.sample('weights', dist.Dirichlet(0.5 * torch.ones(K)))
    with pyro.plate('components', K):
        scales = pyro.sample('scales', dist.LogNormal(0., 2.))
        locs = pyro.sample('locs', dist.Normal(0., 10.))

    with pyro.plate('data', len(data)):
        # Local variables.
        assignment = pyro.sample('assignment', dist.Categorical(weights))
        pyro.sample('obs', dist.Normal(locs[assignment], scales[assignment]), 
                    obs=data)

### Optimizer

In [ ]:
optim = pyro.optim.Adam({'lr': 0.1, 'betas': [0.8, 0.99]})
elbo = TraceEnum_ELBO(max_plate_nesting=1)

Before inference we’ll initialize to plausible values. **Mixture models are very succeptible to local modes**. A common approach is choose the best among many randomly initializations, where the cluster means are initialized from random subsamples of the data. Since we’re using an `AutoDelta` guide, we can initialize by defining a custom `init_loc_fn()`.

In [ ]:
def init_loc_fn(site):
    if site["name"] == "weights":
        # Initialize weights to uniform.
        return torch.ones(K) / K
    if site["name"] == "scales":
        # Initialise std. dev's. to std. dev of data
        return (data.var() * torch.ones(K) / 2).sqrt()
    if site["name"] == "locs":
        # Initialise means to two randomly chosen data points
        return data[torch.multinomial(torch.ones(len(data)) / len(data), K)]
    raise ValueError(site["name"])

def initialize(seed):
    global global_guide, svi
    pyro.set_rng_seed(seed)
    pyro.clear_param_store()
    global_guide = AutoDelta(
        pyro.poutine.block(model, expose=['weights', 'locs', 'scales']), 
        init_loc_fn=init_loc_fn
    )
    svi = SVI(model, global_guide, optim, loss=elbo)
    return svi.loss(model, global_guide, data)

# Choose the best among 100 random initializations.
loss, seed = min((initialize(seed), seed) for seed in range(100))
initialize(seed)
print('seed = {}, initial_loss = {}'.format(seed, loss))

During training, we’ll collect both losses and gradient norms to monitor convergence. We can do this using PyTorch’s `.register_hook()` method.

In [ ]:
# Register hooks to monitor gradient norms.
gradient_norms = defaultdict(list)
for name, value in pyro.get_param_store().named_parameters():
    value.register_hook(
        lambda g, name = name: gradient_norms[name].append(g.norm().item())
    )

losses, params  = [], []
for i in range(200 if not smoke_test else 2):
    loss = svi.step(data)
    losses.append(loss)
    print('.' if i % 100 else '\n', end='')

In [ ]:
# Find lowest point in training
iter_best = np.argmin(losses)
iter_best

In [ ]:
plt.figure(figsize=(10, 3), dpi=100).set_facecolor('white')
plt.plot(losses)
plt.xlabel('iters')
plt.ylabel('loss')
#plt.yscale('log')
plt.title('Convergence of SVI');

In [ ]:
plt.figure(figsize=(10,4), dpi=100).set_facecolor('white')
for name, grad_norms in gradient_norms.items():
    plt.plot(grad_norms, label=name)
plt.xlabel('iters')
plt.ylabel('gradient norm')
plt.yscale('log')
plt.legend(loc='best')
plt.title('Gradient norms during SVI');

In [ ]:
map_estimates = global_guide(data)
weights = map_estimates['weights']
locs = map_estimates['locs']
scales = map_estimates['scales']

## Visualize the mixture model

In [ ]:
nobs, minmax, mean, variance, _, _ = scipy.stats.describe(wp_measurements)

X = np.linspace(*minmax ,101)
Y1 = weights[0].item() * scipy.stats.norm.pdf(
    (X - locs[0].item()) / scales[0].item()
)
Y2 = weights[1].item() * scipy.stats.norm.pdf(
    (X - locs[1].item()) / scales[1].item()
)

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax = axes[0]
ax.hist(wp_measurements, bins=61, density=True)
ax.set_ylabel('Probability density')
ax.grid()
ax.set_title('Histogram of Data')

ax = axes[1]
ax.plot(X, Y1, 'r-', label='Y1')
ax.plot(X, Y2, 'b-', label='Y2')
ax.plot(X, Y1 + Y2, 'k--', label='Y1+Y2')
#ax.plot(data.data.numpy(), np.zeros(len(data)), 'k*')
ax.set_title('PDFs of two-component mixture model')
ax.set_xlabel('Temperature')
ax.set_ylabel('Probability density')
ax.legend()
ax.grid()

plt.savefig('plots/weather-dist-est.pdf', dpi=300)
plt.show()

print(f"{'Estimates':27s} {'True Parameters':27s} ")
print("-"*80)
print(f"weights = {weights.data.numpy().round(3)}       {true_epsilon = }")
print(f"locs = {locs.data.numpy().round(3)}      true_mean = 0")
true_scales = np.array([true_scale, true_b*true_scale])
print(f"scales = {scales.data.numpy().round(3)}      {true_scales = }")